<a href="https://colab.research.google.com/github/yukilost/googlecolab/blob/main/LearningBasedFluids.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---
TODO
- ハイパーパラメータ管理
- GPU設定
- モデルの保存, 読み込み
- 損失の視覚化
- データセットの解凍の修正
- 高速化
---

---

MEMO
- 要素数が異なる場合はnp.ndarrayにできない
- float32を使用する
- MSEは2次元配列の各要素に対して適用されるため変更が必要

---

# <strong> Graph Neural Networkを用いた非圧縮性流体の教師なし学習 </strong>


## 初期設定
---

ライブラリのインポート

In [1]:
from google.colab import drive
from google.colab import output
import os
import math
import pandas as pd
import numpy as np
from sklearn import neighbors
import torch
import torch.nn as nn
from typing import Tuple

Google Driveのマウント

In [2]:
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


デバイスの設定

In [3]:
device: torch.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("device: {}".format(device))

device: cpu


## Utility
---

In [4]:
def parse_vector(v):
  """
  @fn parse_vector
  @brief 辞書型のベクトルのパース
  @param v[dict] 辞書型のベクトル
  @return [np.ndarray] ベクトル
  """
  return np.array([v["x"], v["y"], v["z"]])

In [5]:
def choice(arr, n):
  """
  @fn choice
  @brief 要素をランダムに抽出する
  @param arr[np.ndarray] 抽出対象とする配列
  @param n[int] 抽出する数
  @retval [np.ndarray] 抽出した配列
  @retval [np.ndarray] 元の配列から抽出した数を除いた配列
  """
  s = np.random.choice(arr, n, replace=False)
  arr = np.setdiff1d(arr, s)
  return s, arr

In [6]:
def split(n, n_train, n_valid, n_test):
  """
  @fn split_data
  @brief インデックスをtrain, valid, testに分割する
  @param n[int] インデックス数
  @param n_train[int] トレーニングデータ数
  @param n_valid[int] バリデーションデータ数
  @param n_test[int] テストデータ数
  @retval [np.ndarray] トレーニングデータに使用するインデックス
  @retval [np.ndarray] バリデーションデータに使用するインデックス
  @retval [np.ndarray] テストデータに使用するインデックス
  """
  assert n >= n_train + n_valid + n_test, "Data is not split correctly."
  s = np.arange(1, n+1) 
  s_train, s = choice(s, n_train)
  s_valid, s = choice(s, n_valid)
  s_test, s = choice(s, n_test)
  return s_train, s_valid, s_test

In [7]:
class Simulation():
  """
  @class Simulation
  @brief シミュレーションで使用する変数の設定
  """
  def __init__(self, path_config):
    """
    @fn __init__
    @brief コンストラクタ
    @param path_config[str] 設定ファイルのパス
    """
    # 設定ファイル読み込み
    config = pd.read_json(path_config)

    # 粒子数
    self.boundary_size = parse_vector(config["Simulation"]["BoundarySize"])
    self.num_boundary_particles = self.boundary_size.prod() - (self.boundary_size-2).prod()
    self.fluid_size = parse_vector(config["Simulation"]["FluidSize"])
    self.num_fluid_particles = self.fluid_size.prod()
    self.num_particles = self.num_boundary_particles + self.num_fluid_particles

    # 流体
    self.particle_mass = config["Simulation"]["ParticleMass"]
    self.rest_density = config["Simulation"]["RestDensity"]
    self.kernel_particles = config["Simulation"]["KernelParticles"]
    self.volume = self.num_fluid_particles * self.particle_mass / self.rest_density;

    # 半径
    self.effective_radius = pow(((3.0*self.kernel_particles*self.volume)/(4.0*self.num_fluid_particles*math.pi)), 1.0/3.0);
    self.kernel_radius = self.effective_radius
    self.particle_radius = pow((math.pi/(6.0*self.kernel_particles)), 1.0/3.0)*self.effective_radius;

    # 境界
    self.wall_min = np.array(2.0*self.particle_radius*np.ones(3))
    self.wall_max = np.array(2.0*self.particle_radius*(self.boundary_size-1))

    # タイムステップ
    self.timesteps = config["Simulation"]["MaxTimestep"]

In [11]:
# kd-treeを用いた近傍粒子探索の実装
# def NearestNeighborSearch(pos: np.ndarray, radius: float):
#   tree: neighbors.KDTree = neighbors.KDTree(pos, leaf_size=2)
#   idx: np.ndarray = tree.query_radius(pos[:], r=radius)
#   return idx

In [25]:
class NearestNeighborSearch():
  """
  @class NearestNeighborSearch
  @brief 近傍粒子探索
  """
  def __init__(self, sim, radius):
    """
    @fn __init__
    @brief コンストラクタ
    @param sim[Simulation] シミュレーション管理変数
    @param radius[float] 有効半径
    """
    self.radius = radius
    self.grid_num = np.ceil(sim.boundary_size * 2.0 * sim.particle_radius // self.radius).astype(int) + 1
    self.grid_num_all = int(self.grid_num.prod())
    self.pos_min = np.zeros(3)
    self.pos_max = 2.0*sim.particle_radius*(sim.boundary_size+1)
    self.hash_index = [0] * sim.num_particles
    self.cell_start = np.zeros((self.grid_num_all), dtype=np.int)
    self.cell_end = np.zeros((self.grid_num_all), dtype=np.int)

  def IsValid(self, pos):
    """
    @fn IsValid
    @brief ハッシュ値が有効かどうか判断する
    @param pos[np.ndarray] 粒子位置
    @return [bool] 有効かどうか
    """
    return all(pos >= self.pos_min) and all(pos < self.pos_max)

  def CalcHash(self, pos):
    """
    @fn CalcHash
    @brief ハッシュ値を計算する
    @param pos[np.ndarray] 粒子位置
    @return [int] ハッシュ値
    """
    r = self.radius

    if not self.IsValid(pos):
      return -1

    p = pos // r
    hash = p[0] + self.grid_num[0]*p[1] + self.grid_num[0]*self.grid_num[1]*p[2]
    return hash.astype(np.int)

  def NeighborGrid(self, pos):
    """
    @fn NighborGrid
    @brief 近傍格子を計算する
    @param pos[np.ndarray] 粒子位置
    @return [np.ndarray] 近傍格子のハッシュ値
    """
    neighbor_hash = []
    for dx in range(-1, 2):
      for dy in range(-1, 2):
        for dz in range(-1, 2):
          neighbor_pos = pos + self.radius*np.array([dx, dy, dz])
          hash = self.CalcHash(neighbor_pos) 
          if hash == -1:
            continue
          neighbor_hash.append(hash)
    return np.array(neighbor_hash, dtype=np.int)

  def Set(self, pos):
    """
    @fn Set
    @brief 粒子を格子に登録する
    @param pos[np.ndarray] 粒子位置
    """
    n = pos.shape[0]
    self.hash_index = [0]*n
    for i, p in enumerate(pos):
      hash = self.CalcHash(p)
      self.hash_index[i] = (hash, i)
    sorted_hash_index = np.array(sorted(self.hash_index))
    self.hash_index = sorted_hash_index

    self.cell_start[:] = 0xffffffff
    self.cell_end[:] = 0xffffffff

    # ハッシュ値が0以上の場所
    if self.hash_index[n-1][0] == -1:
      return
    idx = self.hash_index > (-1, -1)
    idx = np.all(idx, axis=1)
    idx = np.where(idx)[0][0]

    # 特定ハッシュ値の始点と終点を記録
    self.cell_start[self.hash_index[idx][0].astype(np.int)] = idx
    for i in range(idx+1, n):
      if self.hash_index[i][0] != self.hash_index[i-1][0]:
        self.cell_end[self.hash_index[i-1][0].astype(np.int)] = i
        self.cell_start[self.hash_index[i][0].astype(np.int)] = i
    self.cell_end[self.hash_index[n-1][0].astype(np.int)] = n

  def Search(self, pos):
    """
    @fn Search
    @brief 近傍粒子探索を行う
    @param pos[np.ndarray] 粒子位置
    @return [np.ndarray] 近傍粒子のインデックス
    """
    n = pos.shape[0]
    res = []
    for p in pos:
      adj = []
      neighbor_grid = self.NeighborGrid(p)
      # 近傍格子のハッシュ値
      for hash in neighbor_grid:
        for i in range(self.cell_start[hash].astype(np.int), self.cell_end[hash].astype(np.int)):
          # 近傍格子内に存在する粒子のインデックス
          idx = self.hash_index[i][1].astype(np.int)
          d = np.linalg.norm(p-pos[idx])
          if d <= self.radius:
            adj.append(idx)
      res.append(np.array(adj))
    return np.array(res, dtype=object)

In [9]:
class Graph():
  """
  @class Graph
  @brief グラフ構造
  """
  def __init__(self, v, e, g):
    """
    @fn __init__
    @brief コンストラクタ
    @param v[torch.tensor] 頂点の特徴量
    @param e[torch.tensor] 辺の特徴量
    @param g[torch.tensor] グローバル特徴量
    """
    self.v = v
    self.e = e
    self.g = g
  
  def __iadd__(self, other):
    """
    @fn __iadd__
    @brief +=のオーバロード
    @param other[Graph] グラフ
    @return [Graph] グラフ
    """
    self.v += other.v
    self.e += other.e
    self.g += other.g
    return self

In [10]:
class FluidInfo():
  """
  @class FluidInfo
  @brief 流体の状態
  """
  def __init__(self, df):
    """
    @fn __init__
    @brief コンストラクタ
    @param df 特定のタイムステップのデータ
    """
    self.pos = df[["position.x", "position.y", "position.z"]].values
    self.vel = df[["velocity.x", "velocity.y", "velocity.z"]].values
    self.acc = df[["acc.x", "acc.y", "acc.z"]].values
    self.vol = df[["volume"]].values

In [12]:
def MatchEdge(idx):
  """
  @fn MatchEdge
  @brief エッジの対応付けを行う
  @param idx[np.ndarray] 近傍粒子のインデックス
  @retval [np.ndarray] 入頂点
  @retval [np.ndarray] 出頂点
  """
  receivers = np.zeros(0, dtype=np.int) 
  senders = np.zeros(0, dtype=np.int)
  for receiver, neighbor in enumerate(idx):
    for sender in neighbor:
      if sender == receiver: 
        continue
      receivers = np.append(receivers, receiver)
      senders = np.append(senders, sender)
  return receivers, senders

## データセット
---

データセットの解凍

In [13]:
!unzip /content/drive/MyDrive/LearningBasedFluids/data.zip -d /content/graph_neural_network/

# 出力消去
output.clear()

Archive:  /content/drive/MyDrive/LearningBasedFluids/data.zip
   creating: /content/graph_neural_network/data/
   creating: /content/graph_neural_network/data/20/
  inflating: /content/graph_neural_network/data/20/20.json  
  inflating: /content/graph_neural_network/data/20/20.csv  
   creating: /content/graph_neural_network/data/18/
  inflating: /content/graph_neural_network/data/18/18.csv  
  inflating: /content/graph_neural_network/data/18/18.json  
   creating: /content/graph_neural_network/data/27/
  inflating: /content/graph_neural_network/data/27/27.json  
  inflating: /content/graph_neural_network/data/27/27.csv  
   creating: /content/graph_neural_network/data/9/
  inflating: /content/graph_neural_network/data/9/9.csv  
  inflating: /content/graph_neural_network/data/9/9.json  
   creating: /content/graph_neural_network/data/11/
  inflating: /content/graph_neural_network/data/11/11.json  
  inflating: /content/graph_neural_network/data/11/11.csv  
   creating: /content/graph_n

## ネットワーク
---

In [22]:
dim = 128

class GraphNeuralNetwork(nn.Module):
  """
  @class GraphNeuralNetwork
  @brief グラフニューラルネットワーク
  """
  def __init__(self):
    """
    @fn __init__
    @brief コンストラクタ
    """
    super().__init__() 

    # Encoder
    self.epsilon_v = nn.Sequential(
        nn.Linear(in_features=10, out_features=dim), nn.ReLU(), 
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(),
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU()
    )
    self.epsilon_e = nn.Sequential(
        nn.Linear(in_features=4, out_features=dim), nn.ReLU(), 
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(),
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(),
    )

    # Processor
    self.phi_v = nn.Sequential(
        nn.Linear(in_features=dim*2+3, out_features=dim), nn.ReLU(), 
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(),
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU()
    )
    self.phi_e = nn.Sequential(
        nn.Linear(in_features=dim*3, out_features=dim), nn.ReLU(), 
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(),
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU()
    )

    # Decoder
    self.delta_v = nn.Sequential(
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(), 
        nn.Linear(in_features=dim, out_features=dim), nn.ReLU(), 
        nn.Linear(in_features=dim, out_features=3)
    )

  def forward(self, sim, info, nns):
    """
    @fn forward
    @brief 推論
    @param sim[Simulation] シミュレーション環境
    @param info[FluidInfo] 流体の状態
    @return [torch.tensor] 加速度
    """
    graph, receivers, senders = self.encoder(sim, info, nns)
    graph += self.processor(graph, receivers, senders)
    out = self.decoder(graph)
    return out

  def encoder(self, sim, info, nns):
    """
    @fn encoder
    @brief グラフの構築
    @param sim[Simulation] シミュレーション環境
    @param info[FluidInfo] 流体の状態
    @retval [Graph] 潜在空間上のグラフ
    @retval [np.ndarray] 入頂点
    @retval [np.ndarray] 出頂点
    """
    # 近傍粒子探索
    nns.Set(info.pos)
    idx = nns.Search(info.pos)
    receivers, senders = MatchEdge(idx)

    # 頂点の特徴量
    wall_dist_min = abs(sim.wall_min - info.pos)
    wall_dist_max = abs(sim.wall_max - info.pos)

    v = np.concatenate((info.vel, info.vol, wall_dist_min, wall_dist_max), axis=1).astype(np.float32)
    v = torch.from_numpy(v).clone()
    v = self.epsilon_v(v)
  
    # 辺の特徴量
    relative_pos = info.pos[senders] - info.pos[receivers]
    magnitude = np.linalg.norm(relative_pos, axis=1).reshape(-1, 1)
    e = np.hstack((relative_pos, magnitude)).astype(np.float32)
    e = torch.from_numpy(e).clone()
    e = self.epsilon_e(e)

    # グラフの特徴量
    g = np.array([0.0, -9.8, 0.0], dtype=np.float32)
    g = torch.from_numpy(g).clone()

    # グラフ構築
    graph = Graph(v, e, g)

    return graph, receivers, senders

  def processor(self, graph, receivers, senders):
    """
    @fn processor
    @brief 特徴量の更新
    @param graph[Graph] 潜在空間上のグラフ
    @param receivers[np.ndarray] 入頂点
    @param senders[np.ndarray] 出頂点
    @retval [Graph] グラフ
    """
    # 辺の更新
    e = torch.cat((graph.e, graph.v[receivers], graph.v[senders]), axis=1)
    e = self.phi_e(e)
    
    # 頂点の更新
    agg_e = self.aggregate(receivers, graph.e, graph.v.shape)
    glo = graph.g.expand((graph.v.shape[0], graph.g.shape[0]))
    v = torch.cat((graph.v, agg_e, glo), axis=1)
    v = self.phi_v(v)

    # グローバル特徴量の更新
    g = torch.zeros(3)

    return Graph(v, e, g)

  def decoder(self, graph):
    """
    @fn decoder
    @brief 出力の抽出
    @param graph[Graph] 潜在空間上のグラフ
    @return [torch.tensor] 加速度
    """
    out = self.delta_v(graph.v)
    return out

  def aggregate(self, receivers, e, shape):
    """
    @fn aggregate
    @brief 特徴量の集約
    @param receivers[np.ndarray] 入頂点
    @param e[torch.tensor] 辺の特徴量
    @param shape[tuple] 頂点数と次元
    @return [torch.tensor] 集約した特徴量
    """
    val = torch.zeros(shape)
    for i in range(shape[0]):
      agg = e[receivers==i]
      val[i] = torch.mean(agg, axis=0)
    return val

## 学習
---

In [23]:
path = "/content/graph_neural_network/processed/"
criterion = nn.MSELoss()
model = GraphNeuralNetwork()
optimizer = torch.optim.Adam(model.parameters())

# データセット分割
train_scenes, valid_scenes, test_scenes = split(30, 20, 5, 5)

def train(model):
  """
  @fn train
  @brief トレーニング
  @param model[GraphNeuralNetwork] 学習するモデル
  """
  model.train()
  for scene in train_scenes:
    print("scene: {}".format(scene))
    # 設定ファイル読み込み
    sim = Simulation(path+"{}/{}.json".format(scene, scene))
    # シーン読み込み
    df = pd.read_csv(path+"{}/{}.csv".format(scene, scene))
    # 流体粒子ラベルの作成
    label_fluid = np.array([False]*(sim.num_particles-sim.num_fluid_particles)+[True]*(sim.num_fluid_particles))
    # 学習用パラメータの設定, Rなど?
    radius = 0.08
    # 近傍粒子探索の設定
    nns = NearestNeighborSearch(sim, radius)
    for timestep in range(1, sim.timesteps):
      print("timestep: {}".format(timestep))
      optimizer.zero_grad()
      dft = df[df["timestep"]==timestep]
      info = FluidInfo(dft)
      y = torch.tensor(info.acc[label_fluid]).float()
      pred = model(sim, info, nns)[label_fluid]
      loss = 3*criterion(y, pred) # MSELossの次元の都合で3をかける
      print("loss: {}".format(loss.item())) 
      loss.backward()
      optimizer.step()

def valid(model):
  """
  @fn valid
  @brief バリデーション
  @param model[GraphNeuralNetwork] 評価するモデル
  """
  model.valid()
  with torch.no_grad():
    for scene in valid_scenes:
      # 設定ファイル読み込み
      sim: Simulation = Simulation(path+"{}/{}.json".format(scene, scene))
      # シーン読み込み
      df: pd.DataFrame = pd.read_csv(path+"{}/{}.csv".format(scene, scene))
      # 流体粒子ラベルの作成
      label_fluid = df[df["timestep"]==1]["label"].values == 1
      # 学習用パラメータの設定, Rなど?
      for timestep in range(sim.timesteps):
        optimizer.zero_grad()
        dft = df[df["timestep"]==timestep]
        acc = model()
        loss = criterion()

In [ ]:
epochs = 10

for epoch in range(epochs):
  print("-"*50)
  print("epoch: {}/{}".format(epoch+1, epochs))
  train(model)
  #valid(model)

--------------------------------------------------
epoch: 1/10
scene: 6
timestep: 1
loss: 105.21780395507812
timestep: 2
loss: 98.59842681884766
timestep: 3
loss: 94.06672668457031
timestep: 4
loss: 93.6860122680664
timestep: 5
loss: 94.85629272460938
timestep: 6
loss: 93.76306915283203
timestep: 7
loss: 90.25790405273438
timestep: 8
loss: 86.13849639892578
timestep: 9
loss: 82.12525939941406
timestep: 10
loss: 77.6533432006836
timestep: 11
loss: 75.87627410888672
timestep: 12
loss: 665.7382202148438
timestep: 13
loss: 418.6941833496094
timestep: 14
loss: 778.4749755859375
timestep: 15
loss: 828.086669921875
timestep: 16
loss: 702.0244750976562
timestep: 17
loss: 674.5125732421875
timestep: 18
loss: 465.3511047363281
timestep: 19
loss: 417.28717041015625
timestep: 20
loss: 393.78656005859375
timestep: 21
loss: 659.0706176757812
timestep: 22
loss: 868.3950805664062
timestep: 23
loss: 1512.86767578125
timestep: 24
loss: 1314.1661376953125
timestep: 25
loss: 1377.125244140625
timestep: 26

In [ ]:
plt